In [20]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [6]:
server = 'mavagha-dv1-sqls.database.windows.net'
database = 'mavaghasqld'
username = 'SicomAdminUser'
password = 'MUdNRsrTl6eF5KjzlVcr'
driver = '{ODBC Driver 18 for SQL Server}'

In [22]:
try:
    conn = pyodbc.connect(
        f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    )
    print("Connexion réussie à SQL Server")
    
    cursor = conn.cursor()
    cursor.execute(
            """SELECT 
                  NUMERO_TRAIN
                  ,CATEGORIE_TRAIN
                  ,FORMAT(REAL_DT_FROM, 'yyyy-MM-dd') AS "DATE DE DEPART"
                  ,FORMAT(REAL_DT_TO, 'yyyy-MM-dd') AS "DATE D'ARRIVEE"
            	  ,FORMAT(REAL_DT_FROM, 'HH:mm') AS "HEURE DE DEPART REEL"
            	  ,FORMAT(REAL_DT_TO, 'HH:mm') AS "HEURE D'ARRIVEE REEL"
                  FORMAT(FLOOR(REAL_TEMPS_PARC_H_EN_GARE) + (REAL_TEMPS_PARC_H_EN_GARE -FLOOR(REAL_TEMPS_PARC_H_EN_GARE)),'00.00') AS "TEMPS PASSER EN GARE"
            FROM tcs.SP_ERAMET_VTRAIN_MINI
            WHERE CATEGORIE_TRAIN IN ('Voyageurs express', 'Voyageurs omnibus')
            AND FORMAT(REAL_DT_TO, 'yyyy-MM-dd') BETWEEN '2024-12-01' AND '2024-12-31'
            """
        )
    data = []
    results = cursor.fetchall()
    for i in results:
        donnees = (i[0], i[1], i[2], i[3], i[4], i[5], i[6])
        data.append(donnees)
    columns = [column[0] for column in cursor.description]

    df_inforail = pd.DataFrame(data, columns=["N° TRAIN", "TYPE DE TRAIN", "DATE DE DEPART", "DATE D'ARRIVEE", "HEURE DE DEPART REEL", "HEURE D'ARRIVEE REEL", "TEMPS PASSER EN GARE"])
    print(df_inforail)

    fig, ax = plt.subplots(figsize=(14, 7))
    ax.axis('tight')
    ax.axis('off')

    ax.set_title("PROGRAMME DES TRAINS EN COURS", y=0.9)
    #table = ax.table(cellText=df_express.values, colLabels=df_express.columns, cellLoc='center', loc='center')
    ax.table(cellText=df_inforail.values, colLabels=df_inforail.columns, cellLoc='center', loc='center')

    cursor.close()
    conn.close()
except Exception as e:
    print(f"Erreur de connexion : {e}")

Connexion réussie à SQL Server
Erreur de connexion : ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near 'FORMAT'. (102) (SQLExecDirectW)")
